In [1]:
from py.race_function import scrape_current_race, scrape_current_race_no_odds
from catboost import CatBoostRanker
import pandas as pd

In [2]:
def prediction(race_url, odds_url):
    model = CatBoostRanker()
    model.load_model('model/catboost_ranking_model.cbm')

    df = scrape_current_race(race_url, odds_url)
    X = df.iloc[:, :-1]
    predictions = model.predict(X)

    rated_scores = pd.DataFrame(predictions, columns = ['score'])

    name = df.iloc[:, -1]

    # Combine horse names and predicted scores into one DataFrame
    predicted = pd.concat([name.reset_index(drop=True), rated_scores.reset_index(drop=True)], axis=1)

    # Optionally rename columns if name column has no name
    predicted.columns = ['Horse_name', 'Predicted_Score']

    predicted = predicted.sort_values('Predicted_Score', ascending=False)

    return predicted

In [3]:
def prediction_no_odds(race_url):
    model = CatBoostRanker()
    model.load_model('model/catboost_ranker.cbm')

    df = scrape_current_race_no_odds(race_url)
    X = df.iloc[:, :-1]
    predictions = model.predict(X)

    rated_scores = pd.DataFrame(predictions, columns = ['score'])

    name = df.iloc[:, -1]

    # Combine horse names and predicted scores into one DataFrame
    predicted = pd.concat([name.reset_index(drop=True), rated_scores.reset_index(drop=True)], axis=1)

    # Optionally rename columns if name column has no name
    predicted.columns = ['Horse_name', 'Predicted_Score']

    predicted = predicted.sort_values('Predicted_Score', ascending=False)

    return predicted

In [10]:
def prediction_no_odds_new(race_url):
    model = CatBoostRanker()
    model.load_model('model/catboost_ranker_1.cbm')

    df = scrape_current_race_no_odds(race_url)
    X = df.iloc[:, :-1]
    predictions = model.predict(X)

    rated_scores = pd.DataFrame(predictions, columns = ['score'])

    name = df.iloc[:, -1]

    # Combine horse names and predicted scores into one DataFrame
    predicted = pd.concat([name.reset_index(drop=True), rated_scores.reset_index(drop=True)], axis=1)

    # Optionally rename columns if name column has no name
    predicted.columns = ['Horse_name', 'Predicted_Score']

    predicted = predicted.sort_values('Predicted_Score', ascending=False)

    return predicted

In [17]:
race_url = 'https://racing.hkjc.com/racing/information/English/racing/RaceCard.aspx?RaceDate=2025/09/10&Racecourse=HV&RaceNo=8'
odds_url = 'https://bet.hkjc.com/en/racing/wp/2025-09-10/HV/8'

# df = prediction(race_url, odds_url)
df = prediction(race_url, odds_url)
df1 = prediction_no_odds(race_url)
df2 = prediction_no_odds_new(race_url)

In [18]:
df

,Horse_name,Predicted_Score
5,STORMING DRAGON,1.063786
2,SOVEREIGN FUND,0.874724
9,KING OF FIGHTERS,0.624747
0,DRAGON FOUR SEAS,0.126276
11,MONARCH COUNTY,-0.098450
8,VICTOR THE RAPID,-0.254325
4,REGAL GEM,-0.294512
3,GIANT LEAP,-0.401168
10,THRIVING BROTHERS,-0.571141
1,POWER KOEPP,-0.572337


In [19]:
df1

,Horse_name,Predicted_Score
5,STORMING DRAGON,0.579560
11,MONARCH COUNTY,0.573378
3,GIANT LEAP,0.135330
2,SOVEREIGN FUND,0.049979
1,POWER KOEPP,-0.067203
0,DRAGON FOUR SEAS,-0.089463
9,KING OF FIGHTERS,-0.115060
10,THRIVING BROTHERS,-0.549415
7,LUCKY PLANET,-0.761189
8,VICTOR THE RAPID,-1.000591


In [20]:
df2

,Horse_name,Predicted_Score
5,STORMING DRAGON,0.741216
11,MONARCH COUNTY,0.522000
2,SOVEREIGN FUND,0.184918
3,GIANT LEAP,0.166472
0,DRAGON FOUR SEAS,-0.010210
1,POWER KOEPP,-0.012245
9,KING OF FIGHTERS,-0.089332
10,THRIVING BROTHERS,-0.519442
7,LUCKY PLANET,-0.748513
8,VICTOR THE RAPID,-0.882446
